In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, TextSplitter

text_splitter = RecursiveCharacterTextSplitter(
         chunk_size=3000,
         chunk_overlap=500,
         length_function=len,
         add_start_index=True,
      )


In [11]:
from langchain_community.document_loaders import DedocPDFLoader

file_path = "data\clinrec\КР223.pdf"
loader = DedocPDFLoader(file_path, language='rus+eng')
documents = loader.load_and_split(text_splitter)


In [23]:
PROMPT_TEMPLATE = """
Пожалуйста, перефразируйте следующий текст, удаляя ненужные символы (например, "**", "#", и т.д.) и лишние детали, 
при этом сохраняя всю основную информацию без потери смысла. Так как это медицинский документ, важно быть максимально точным и сохранить исходный смысл.
В ответе запиши только исправленный текст
{context}
"""

In [13]:
import argparse
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain_community.vectorstores import Chroma



def get_fixed_text(query_text):
   
   prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
   prompt = prompt_template.format(context=query_text)
   print(f"Полученный промпт {prompt}")

   # Подключение к LM Studio и отправка запроса
   model = ChatOpenAI(temperature=0.7, base_url="http://localhost:1234/v1", api_key="not-needed")
   response_text = model.predict(prompt)

   # Выводим результаты ответа
   formatted_response = f"Ответ: {response_text}\n"
   print(formatted_response)
   return response_text

In [24]:
get_fixed_text(documents[20].page_content)

Полученный промпт Human: 
Пожалуйста, перефразируйте следующий текст, удаляя ненужные символы (например, "**", "#", и т.д.) и лишние детали, 
при этом сохраняя всю основную информацию без потери смысла. Так как это медицинский документ, важно быть максимально точным и сохранить исходный смысл.
В ответе запиши только исправленный текст
протеинурии.

Применение комбинированной химиорадиотерапии не только не улучшает среднюю

выживаемость пациентов по сравнению с «консервативной» терапией, а даже

увеличивает частоту рецидивов, поэтому предпочтительно использовать

наружную терапию, биологические препараты или их комбинацию [49].

Рекомендуется у пациентов с грибовидным микозом на поздних стадиях с

агрессивным течением и/или неэффективностью других методов лечения

применение в качестве системной химиотерапии монотерапия: Я#гемцитабином** по

схеме 1200 мг/м? в 1, 8 и 15-й дни 28-дневного цикла, всего 6 циклов [53]

Уровень убедительности рекомендаций С (уровень достоверности

доказатель

'Рецидивы грибовидного микоза могут быть вызваны комбинированной химиорадиотерапией. Поэтому предпочтительно использовать наружную терапию, биологические препараты или их комбинацию.\n\nДля пациентов с агрессивным течением грибовидного микоза на поздних стадиях, неэффективных других методов лечения, рекомендуется применение системной химиотерапии монотерапии:\n\n- Гемцитабином в дозе 1200 мг/м² в 1, 8 и 15-й дни 28-дневного цикла, всего 6 циклов.\n- Гемцитабин в малых дозах с продолжительной инфузией в дозе 250 мг/м² в течение 6 часов 1 раз в неделю.\n\nДля взрослых пациентов с экспрессией CD30 + антигена, не достигших ответа после минимум одной линии предшествующей системной терапии, рекомендуется брентуксимаб ведотин.\n\nДля пациентов с прогрессирующими формами грибовидного микоза, рефрактерными как минимум к 5 линиям системной терапии, возможно применение непрограммных схем лечения с паллиативной целью:\n\n- Яциклофосфамид (50-100 мг внутрь ежедневно, не более 1 года терапии).\n- Яэ

In [25]:
fixed_documents = []
for document in documents[:2]:
    fixed_documents.append(get_fixed_text(document.page_content))

Полученный промпт Human: 
Пожалуйста, перефразируйте следующий текст, удаляя ненужные символы (например, "**", "#", и т.д.) и лишние детали, 
при этом сохраняя всю основную информацию без потери смысла. Так как это медицинский документ, важно быть максимально точным и сохранить исходный смысл.
В ответе запиши только исправленный текст
у ipepmyenu. J аберждено: J 1верждено;

Общероссийская общественная Региональная общественная Ассоциация содействия развитию

организация «Российское организация «Общество гематологии и трансплантологии

дерматовенерологов и онкогематологов» костного мозга «Национальное

косметологов» BORG .№-1. от 16.01.2023 г. гематологическое общество»

 

Грибовидный микоз

Кодирование по Международной (84.0

статистической классификации

болезней и проблем, связанных

со здоровьем:

Возрастная группа: Дети/взрослые

Год утверждения: 2023

Разработчик клинической рекомендации:

e Общероссийская общественная организация «Российское общество

дерматовенерологов и космет

In [36]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def get_embeddings():
   model_kwargs = {'device': 'cuda'}
   embeddings_hf = HuggingFaceEmbeddings(
       model_name='intfloat/multilingual-e5-large',
       model_kwargs=model_kwargs
   )
   return embeddings_hf

In [37]:
from langchain_community.vectorstores import FAISS

faiss_index = FAISS.from_documents(documents, get_embeddings())
docs = faiss_index.similarity_search("Что такое грибовидный микоз?", k=2)
for doc in docs:
    print(str(doc.metadata) + ":", doc.page_content)


2024-11-08 00:10:08,285 - c:\Users\kopys\.conda\envs\ollama-pdf\lib\site-packages\sentence_transformers\SentenceTransformer.py - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-large
{'uid': 'doc_uid_auto_def52884-9d46-11ef-ac01-00a61830317a', 'file_name': 'КР223.pdf', 'temporary_file_name': '1731011132_492.pdf', 'size': 21256777, 'modified_time': 1731011132, 'created_time': 1731011132, 'access_time': 1731011318, 'file_type': 'application/pdf', 'rotated_page_angles': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], 'producer': 'PyPDF2', 'start_index': 82579}: у врача- терапии до

дерматовенеролога регресса

и онкогематолога заболавания

48

Приложение В. Информация для пациента

1. Грибовидный микоз относят к злокачественным заболеваниям кожи, с

относительно благоп